# 다각형분류 CNN
- [컨볼루션 신경망 모델 만들어보기](https://tykimos.github.io/2017/03/08/CNN_Getting_Started/)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp /content/drive/MyDrive/Colab\ Notebooks/datasets/handwriting/handwriting_shape.zip .
!cp /content/drive/MyDrive/Colab\ Notebooks/datasets/handwriting/hard_handwriting_shape.zip .

# unzipping compressed file
!unzip -qq handwriting_shape.zip -d handwriting_shape
!unzip -qq hard_handwriting_shape.zip -d hard_handwriting_shape

replace handwriting_shape/test/circle/circle016.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace handwriting_shape/test/circle/circle017.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace handwriting_shape/test/circle/circle018.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace handwriting_shape/test/circle/circle018.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace handwriting_shape/test/circle/circle019.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace handwriting_shape/test/circle/circle020.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace handwriting_shape/test/rectangle/rectangle016.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace handwriting_shape/test/rectangle/rectangle017.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace hard_handwriting_shape/test/circle/circle021.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## 1. CNN 모델

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# seed 값 설정
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터셋 생성하기

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [5]:
!ls handwriting_shape/train

circle	rectangle  triangle


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [7]:
!ls handwriting_shape/test

circle	rectangle  triangle


In [8]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [9]:
train_generator.filenames[0]

'circle/circle001.png'

## 모델 정의/설정/학습

In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

## 모델 학습시키기

케라스에서는 모델을 학습시킬 때 주로 fit() 함수를 사용하지만 제네레이터로 생성된 배치로 학습시킬 경우에는 fit_generator() 함수를 사용합니다. 본 예제에서는 ImageDataGenerator라는 제네레이터로 이미지를 담고 있는 배치로 학습시키기 때문에 fit_generator() 함수를 사용하겠습니다.

- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 train_generator으로 지정합니다.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정합니다. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정합니다.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정합니다. 100번을 반복적으로 학습시켜 보겠습니다.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 validation_generator으로 지정합니다.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정합니다. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정합니다.

In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=50,
    validation_data=test_generator,
    validation_steps=5
)

Epoch 1/50


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


15/15 [==============================] - 2s 31ms/step - loss: 1.0859 - accuracy: 0.5358 - val_loss: 1.5186 - val_accuracy: 0.3333
Epoch 2/50
15/15 [==============================] - 0s 7ms/step - loss: 0.2089 - accuracy: 0.9884 - val_loss: 2.7649 - val_accuracy: 0.4000
Epoch 3/50
15/15 [==============================] - 0s 7ms/step - loss: 0.0136 - accuracy: 1.0000 - val_loss: 3.7904 - val_accuracy: 0.4000
Epoch 4/50
15/15 [==============================] - 0s 6ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 4.5883 - val_accuracy: 0.4000
Epoch 5/50
15/15 [==============================] - 0s 6ms/step - loss: 6.6927e-04 - accuracy: 1.0000 - val_loss: 5.1453 - val_accuracy: 0.4000
Epoch 6/50
15/15 [==============================] - 0s 7ms/step - loss: 4.1056e-04 - accuracy: 1.0000 - val_loss: 5.3565 - val_accuracy: 0.4000
Epoch 7/50
15/15 [==============================] - 0s 6ms/step - loss: 2.9946e-04 - accuracy: 1.0000 - val_loss: 5.3788 - val_accuracy: 0.4000
Epoch 8/50
15/15 [

## 모델 평가

In [13]:
print('-- Evaluate Generator--')
scores = model.evaluate_generator(test_generator, steps=5)
print('%s: %.2f%%' % (model.metrics_names[1], scores[1]*100))

-- Evaluate Generator--
accuracy: 40.00%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
